In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, Textarea
from ipywidgets import Button, GridBox, Layout, ButtonStyle, Box, Label, Dropdown, FloatText, HBox, VBox
import ipywidgets as widgets
from IPython.display import display_html, clear_output

In [2]:
from IPython.display import display
import os
import pandas as pd
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt

In [3]:
from io import BytesIO
import base64
import glob

In [4]:
import plot_for_notebook

In [116]:
class BoxFields:
    def __init__(self, protein_value):
        
        self.func = None 
        self.protein_value = protein_value
        self.ansamble_value = 'NVE'
        self.water_model_value ='spce'
        self.analysis_type_value = 'autocorrelation'
        
        
        self.form_item_layout = Layout(
                                        width= '99%',
                                        align_items='stretch'
                                    )
        self.form_item_layout_box = Layout(
                                        width= '50%',
                                        align_items='stretch',
                                    )
        self.water_model =  widgets.Dropdown(
        options=['spce', 'tip3p', 'tip4p-ew', 'tip4p-d'],
        value='spce',
        description='water model:',
        disabled=False,
        layout=self.form_item_layout_box)

        self.ansamble = widgets.Dropdown(
                options=['NVE', 'NPT_gamma_ln_0.01', 'NPT_gamma_ln_2', 'NVT_gamma_ln_0.01' ],
                value='NVE',
                description='ansamble:',
                layout=self.form_item_layout_box,
            )
        self.protein = widgets.Dropdown(
                options=['ubq', 'h4'],
                value=self.protein_value,
                description='protein:',
                layout=self.form_item_layout,
                dropdown_color='moccasin',
                align_items='left',
                    border='solid',
                    width='100%'
            )
        
        self.analysis_type = widgets.Dropdown(
                    options=['autocorrelation', 'translational diffusion'],
                    value='autocorrelation',
                    layout=Layout(width='99%', grid_area='header'),
                    align_items='center',
                            border='solid',
                            width='99%')

        
        
    def get_form(self):

        self.form_items = [self.protein, 
                           VBox([self.water_model, self.ansamble])]

        self.form = Box(self.form_items, layout=Layout(
                                        display='flex',
                                        flex_flow='column',
                                        border='solid 2px',
                                        align_items='stretch',
                                        width='50%'
                                    ))
        return self.form
    
    def set_func(self, func):
        self.func = func
    
    def on_value_change(self, change):
        return change['owner'].description.strip(':'), change['new']
    
    def main_change(self, change):

        owner_value, new_value = self.on_value_change(change)
        if owner_value == 'protein':
            self.protein_value = new_value
        elif owner_value == 'water model':
            self.water_model_value = new_value
        elif owner_value == 'ansamble':
            self.ansamble_value = new_value
        elif owner_value == 'analysis_type':
            self.analysis_type_value = new_value
        if self.func:
            self.func()
        
        
    def observe(self):
        self.water_model.observe(self.main_change, names='value')
        self.ansamble.observe(self.main_change, names='value')
        self.protein.observe(self.main_change, names='value')
        self.analysis_type.observe(self.main_change, names='value')


In [6]:
class GridBoxFields:
    
    def __init__(self, protein_1, protein_2):
        self.box_fields_1 = BoxFields(protein_1)
        self.box_fields_2 = BoxFields(protein_2)
        
    def set_func(self, func):
        self.func = func
        
    def get_hbox(self):
        self.hbox = HBox([self.box_fields_1.get_form(), self.box_fields_2.get_form()])
        return self.hbox
    
        
    def get_gridbox(self):
        header = self.box_fields_1.analysis_type
        main = self.get_hbox()


        gridbox = GridBox(children=[header, main],
                layout=Layout(
                    grid_template_areas='''
                    "header"
                    "main "
                    ''')
               )
        return gridbox
    def observe(self):
        if self.func:
            self.func()
        self.box_fields_1.observe()
        self.box_fields_2.observe()

In [28]:
class ImageDisplay:
    def __init__(self, protein_1, protein_2):
        self.gridbox = GridBoxFields(protein_1, protein_2)
        self.numbers = 0
        self.order = 0
        
    def set_numbers(self, numbers):
        self.numbers = numbers
        
    def set_order(self, order):
        self.order = order  
        
    def get_figure_str(self,analysis="autocorrelation", group="NH", 
                       numbers=1, order=2, box_fields='box_fields_1'):
        
        choice_box = {'box_fields_1': image_display.gridbox.box_fields_1,
                      'box_fields_2': image_display.gridbox.box_fields_2}
        analysis=choice_box[box_fields].analysis_type_value
        ansamble=choice_box[box_fields].ansamble_value
        water_model=choice_box[box_fields].water_model_value
        protein=choice_box[box_fields].protein_value
        choice_analysis_path = {"autocorrelation": os.path.join(protein, water_model, ansamble,\
                          "autocorr", group, "data"),
               "translational_diffusion": os.path.join(protein, water_model, ansamble,\
                          analysis),
               "overall_thumbling": os.path.join(protein, water_model, ansamble,\
                          analysis)}
        data_path =  choice_analysis_path[analysis]
        try:
            csv_files = sorted(glob.glob(os.path.join(data_path,"acorr", "*.csv")))
            if numbers > len(csv_files):
                numbers = len(csv_files) - 1
            path_to_accor = csv_files[numbers]
            path_to_fit = os.path.join(data_path,"fit")
            fig, ax = plot_for_notebook.get_plot_acorr_fit(path_to_fit, path_to_accor, order)
            blob = BytesIO()
            plt.savefig(blob,format="png")
            plt.close()
            self.img1_base64 = base64.b64encode(blob.getvalue()).decode("utf-8") 
            return self.img1_base64
        except IndexError:
            print("Trajectory {protein} {ansamble} {water_model} isn't ready for handling".\
                  format(protein=protein, ansamble=ansamble, water_model=water_model))
        
    
    def show_image(self, numbers=1, order=2):
        
        if self.numbers:
            numbers = self.numbers
            
        if self.order:
            order = self.order
 
        for ind in range(numbers):
                display_html("""

            <table style="width:100%">
              <tr>
                <th></th>
                <th></th>
              </tr>
              <tr>
                <td><img src="data:image/png;base64, {img}"></td>
                <td><img src="data:image/png;base64, {img1}"></td>
              </tr>
            </table> 

            """.format(img=self.get_figure_str(box_fields='box_fields_1', numbers=ind, order=order), 
                       img1=self.get_figure_str(box_fields='box_fields_2', numbers=ind, order=order)),
                       raw=True)

In [119]:
numbers_of_picture = widgets.IntText(description="numbers")
order_exp = widgets.Dropdown(
        options=[2, 3, 4],
        value=2,
        description='order:',
        disabled=False,)

display(numbers_of_picture)
display(order_exp)
image_display = ImageDisplay('ubq', 'h4')
display(image_display.gridbox.get_gridbox())

out = Output() 
display(out)


image_display.gridbox.set_func(image_display.show_image)
image_display.gridbox.box_fields_1.set_func(image_display.show_image)
image_display.gridbox.box_fields_2.set_func(image_display.show_image)
image_display.gridbox.observe()


def on_value_change(change):
    global out


    if change['owner'].description.strip(':') == 'order':
        image_display.set_order(change['new']) 
        
    if change['owner'].description.strip(':') == 'numbers':  
        image_display.set_numbers(change['new'])
    with out:
        clear_output(wait=True)
        image_display.gridbox.set_func(image_display.show_image)
        image_display.gridbox.box_fields_1.set_func(image_display.show_image)
        image_display.gridbox.box_fields_2.set_func(image_display.show_image)
        image_display.gridbox.observe()
    clean = 1
    
numbers_of_picture.observe(on_value_change, names='value')
order_exp.observe(on_value_change, names='value')

IntText(value=0, description='numbers')

Dropdown(description='order:', options=(2, 3, 4), value=2)

GridBox(children=(Dropdown(layout=Layout(grid_area='header', width='99%'), options=('autocorrelation', 'transl…

Output()

Trajectory h4 NVE spce isn't ready for handling


,
,
